In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [3]:
relations = []
for data in tqdm(datas):
    for i in data['Parses']:
        relation = i['InferentialChain']
        if relation is not None:
            relations += relation
relations = set(relations)

100%|██████████| 1639/1639 [00:00<00:00, 546851.03it/s]


In [2]:
sparql_all_relations = """
PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?r
WHERE {
?e ?r ?e1 .
}
"""
r = execute_sparql(sparql_all_relations)
r = set([i['r']['value'].replace("http://rdf.freebase.com/ns/", "") for i in r if i['r']['value'].startswith("http://rdf.freebase.com/ns/")])

In [7]:
filter_relations = list(r - relations)

In [1]:
set([i.split('.')[0] for i in r]) - set([i.split('.')[0]  for i in relations])

NameError: name 'r' is not defined

In [5]:
[i for i in r if "description" in i]

['base.aubreymaturin.ship_mention.description',
 'user.patrick.default_domain.room.description',
 'user.philg.acre.acre_api_method.description',
 'base.jewlib.printed_description.contains_information_on',
 'base.modelingexamples.example.model_description',
 'base.schemastaging.phone_sandbox.description',
 'base.fcintcommunity.services.description',
 'base.aubreymaturin.character_mention.description',
 'base.schemastaging.listed_serving_size.quantity_description',
 'base.wakfu.item.description',
 'base.sa3base.sa3_textual.description_for_fr_title',
 'base.sa3base.sa3_textual.description_for_system_name',
 'user.itteco.ga.domain.item.description',
 'award.long_list_nomination.notes_description',
 'base.sa3base.sa3_textual.description_for_mapping_between_view',
 'common.webpage.description',
 'freebase.property_hints.inverse_description',
 'user.xandr.scrap.schema.item.description',
 'base.mapcentral.fgdc_process_step.process_description',
 'base.aubreymaturin.species_mention.description'